# core

> Utils useful for generic data processing


In [ ]:
# | default_exp core.utils

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | export

from __future__ import annotations

import inspect
import logging
import os
from typing import Any, Callable, Container, Iterable, Literal, Mapping, TypeVar

from fastcore.test import test_fail

In [ ]:
# | export
logger = logging.getLogger(__name__)

T = TypeVar("T")
K = TypeVar("K")

PathLike = str | os.PathLike | None
Iterifiable = Iterable[T] | T | None

## Class inspection

> Class and object inspection functions.


In [ ]:
# | export


def get_class_vars(cls: type) -> dict[str, Any]:
    """Get all class variables of a class.

    Parameters
    ----------
    cls : type
        Class to get variables from.

    Returns
    -------
    dict[str, Any]
        Dictionary of class variables.
    """
    return {
        k: v
        for k, v in cls.__dict__.items()
        if not k.startswith("__") and not callable(v)
    }

Example of class vars usage


In [ ]:
from typing import ClassVar


class A:
    a: ClassVar[int] = 1


get_class_vars(cls=A)

{'a': 1}

In [ ]:
# | export


def get_class_name(t: type) -> str:
    """Get the full name of a class inlcuding the module name.

    Parameters
    ----------
    t : type
        The type to get the name of.

    Returns
    -------
    str
        The full name of the type.
    """
    if isinstance(t, type):
        return f"{t.__module__}.{t.__name__}"
    else:
        raise TypeError(f"Expected type, got {type(t)}")


def get_object_class_name(o: object) -> str:
    """Returns the full name of the class of an object.

    Parameters
    ----------
    o : object
        object to get the class name of.

    Returns
    -------
    str
        The full name of the class of the object.
    """
    return get_class_name(o.__class__)

## Utility convertes

> Utility functions for converting between different formats.


In [ ]:
# | export


def iterify(obj: object) -> Iterable:
    """Make an object iterable if it is not already.

    Parameters
    ----------
    obj : object
        Input object to be made iterable if it is not already.

    Returns
    -------
    Iterable
        Iterable version of the input object.
    """ """"""
    if isinstance(obj, Iterable) and not isinstance(obj, str):
        return obj
    else:
        return [obj]

In [ ]:
# | export


def listify(
    obj: object,
    nested_collections: bool = False,
    none_handlings: Literal["none", "empty", "wrap", "default"] = "empty",
    default_value: Any = None,
) -> list | None:
    """Make an object a list if it is not already.

    Parameters
    ----------
    obj : object
        input object to be made a list if it is not already.
    nested_collections : bool, optional
        How to handle the case if input object is already a collection, if True, the input object will be wrapped in a list, if False, the input object will be converted to a list, by default False
    none_handlings : Literal[&#39;none&#39;, &#39;empty&#39;, &#39;wrap&#39;, &#39;default&#39;], optional
        How to handle the case if input object is None, by default &#39;empty&#39;
            - &#39;none&#39;: return None
            - &#39;empty&#39;: return an empty list
            - &#39;wrap&#39;: return a list with None as the only element
            - &#39;default&#39;: return a list with default_value as the only element
    default_value : Any, optional
        Default value to be used if none_handlings is &#39;default&#39;, by default None

    Returns
    -------
    List
        List version of the input object.

    Raises
    ------
    ValueError
        If none_handlings is not one of &#39;none&#39;, &#39;empty&#39;, &#39;wrap&#39;, &#39;default&#39;.
    """
    if isinstance(obj, str):
        return [obj]
    elif isinstance(obj, (set, list, tuple, Mapping)):
        if nested_collections:
            return [obj]
        else:
            return list(obj)
    elif obj is None:
        if none_handlings == "none":
            return None
        elif none_handlings == "empty":
            return []
        elif none_handlings == "wrap":
            return [None]
        elif none_handlings == "default":
            return [default_value]
        else:
            raise ValueError(
                f"Invalid none_handlings: `{none_handlings}`, choose from `none`, `empty`, `wrap`, `default`."
            )
    else:
        return [obj]

In [ ]:
listify(None, none_handlings="wrap")

[None]

In [ ]:
listify(None, none_handlings="none")

In [ ]:
listify([1, 2, 3])

[1, 2, 3]

In [ ]:
listify(1)

[1]

In [ ]:
list({"a": 1, "b": 2})

['a', 'b']

In [ ]:
# | export


def tuplify(obj: object, num: int = 2) -> tuple:
    """Convert an object to a tuple with length num, internally calls listify.

    Parameters
    ----------
    obj : object
        Input object to be converted to a tuple.
    num : int, optional
        Length of collection, by default 2

    Returns
    -------
    tuple
        Tuple version of the input object.
    """

    if isinstance(obj, (tuple, list)):
        if len(obj) == num:
            return tuple(obj)
        elif len(obj) < num:
            return tuple([obj[i] if i < len(obj) else obj[0] for i in range(num)])
        else:
            return obj[:num]
    else:
        return tuple([obj] * num)

In [ ]:
# | export


def dictify_with_names(
    data: Any | list | tuple | dict, default_name="image"
) -> dict[str, Any]:
    """Converts the data to a dictionary with names

    Parameters
    ----------
    data : Any | list | tuple | dict
        data to convert, may be a dictionary of {name: data}, a list, a tuple, list/tuple of 2 lists/tuples (names, datas) or a single object
    default_name : str, optional
        default name for the data that will be enumerated if the data is not a dictionary, by default 'image'

    Returns
    -------
    dict[str, Any]
        a dictionary of {name: data}

    Raises
    ------
    ValueError
        data[0] and data[1] must be the same length, but are {len(data[0])} and {len(data[1])} respectively
    """
    if isinstance(data, dict):
        return data
    elif isinstance(data, (tuple, list)):
        if (
            len(data) == 2
            and isinstance(data[0], (list, tuple))
            and isinstance(data[1], (list, tuple))
        ):
            if len(data[0]) == len(data[1]):
                return {data[0][i]: data[1][i] for i in range(len(data[0]))}
            else:
                raise ValueError(
                    f"data[0] and data[1] must be the same length, but are {len(data[0])} and {len(data[1])} respectively"
                )
        else:
            data = listify(data)
            return {f"{default_name}_{i}": d for i, d in enumerate(data)}
    else:
        data = listify(data)
        return {f"{default_name}_{i}": d for i, d in enumerate(data)}

In [ ]:
dictify_with_names((["a", "b", "c"], [1, 2, 3]))

{'a': 1, 'b': 2, 'c': 3}

In [ ]:
dictify_with_names({"a": 1, "b": 2, "c": 3})

{'a': 1, 'b': 2, 'c': 3}

In [ ]:
dictify_with_names(["a", "b", "c"], default_name="param")

{'param_0': 'a', 'param_1': 'b', 'param_2': 'c'}

In [ ]:
dictify_with_names(None)

{}

In [ ]:
dictify_with_names(data="a")

{'image_0': 'a'}

## Existence handlers


In [ ]:
# | export


def handle_existing(
    obj: K | T,
    existence_predicate: Callable[[T], bool],
    must_exist: bool = True,
    obj_processor: Callable[[K], T] | None = None,
    obj_type_name: str = "object",
    strategy: Literal["skip", "overwrite", "raise"] = "skip",
    error_type: type[Exception] = ValueError,
) -> T | None:
    """Handle existing object in the way specified by strategy.

    Parameters
    ----------
    obj : K | T
        object to be handled, may be key, filename, etc.
    existence_predicate : Callable[[T], bool]
        function that returns True if key  (object) exists, False otherwise
    must_exist : bool, optional
        whether the object must exist or not,
            if False and object does not exist, the strategy is applied,
            if True and object does exist, the strategy is not applied, by default True
    obj_processor : Callable[[K], T] | None, optional
        function that converts object to the key used in predicate,
        if None, obj is used itself, by default None
    obj_type_name : str, optional
        name of the object type, that will be used in logging, by default "object"
    strategy : Literal['skip', 'overwrite', 'raise'], optional
        how to handle existing object:
        - "skip": skip the object (return None)
        - "overwrite": overwrite the object (return path)
        - "raise": raise an error (raise ExistsError)
        (default: "skip")
    error_type : type[Exception], optional
        type of error to raise if strategy is "raise", by default ValueError

    Returns
    -------
    K | T | None:
        if object must be skipped, returns None, if overwrite or object does not exist, returns object

    Raises
    ------
    ValueError
        if existing_handling is not one of "skip", "overwrite", or "raise"
    """
    obj: T = obj_processor(obj) if obj_processor else obj  # type: ignore
    exists = not existence_predicate(obj) if must_exist else existence_predicate(obj)
    if exists:
        msg = f"{obj_type_name.capitalize()} `{str(obj)}` {'does not exist' if must_exist else 'already exists'}"
        if strategy == "skip":
            logger.info(f"{msg}, skipping")
            return None
        elif strategy == "overwrite":
            logger.warning(f"{msg}, {'creating' if must_exist else 'overwriting'}")
            return obj
        elif strategy == "raise":
            raise error_type(f"{msg}")
        else:
            raise ValueError(f"Unknown existing_handling strategy: `{strategy}`")
    else:
        return obj


def hadnle_existing_key(
    key: K,
    container: Container,
    must_exist: bool = True,
    strategy: Literal["skip", "overwrite", "raise"] = "skip",
) -> K | None:
    """Handle existing key in the way specified by strategy.

    Parameters
    ----------
    key : K
        key to be handled
    container : Container
        container to check if key exists in
    must_exist : bool, optional
        whether the object must exist or not, if False and object does not exist the strategy is applied, by default True
    strategy : Literal['skip', 'overwrite', 'raise'], optional
        how to handle existing key:
        - "skip": skip the key (return None)
        - "overwrite": overwrite the key (return key)
        - "raise": raise an error (raise ExistsError)
        (default: "skip")

    Returns
    -------
    K | None:
        if key must be skipped, returns None, if overwrite or key does not exist, returns key

    Raises
    ------
    KeyError
        if key exists and existing_handling is "raise"
    ValueError
        if existing_handling is not one of "skip", "overwrite", or "raise"
    """
    return handle_existing(
        key,
        lambda k: k in container,
        must_exist=must_exist,
        obj_type_name="key",
        strategy=strategy,
        error_type=KeyError,
    )

In [ ]:
d_with = {"a": 2}
handle_existing(
    "a",
    lambda x: x in d_with,
    must_exist=False,
    obj_type_name="key",
    strategy="overwrite",
)

Key `a` already exists, overwriting


'a'

In [ ]:
d_without = {}
handle_existing(
    "a",
    lambda x: x in d_without,
    must_exist=True,
    obj_type_name="key",
    strategy="overwrite",
)

Key `a` does not exist, creating


'a'

In [ ]:
hadnle_existing_key("a", d_with, must_exist=False, strategy="skip")

In [ ]:
hadnle_existing_key("a", d_without, must_exist=True, strategy="skip")

## Arguments processing


In [ ]:
# | export


def exclusive_args(
    exclusive_arg_names: Iterifiable[str] = [],
    may_be_empty: bool = False,
    empty_value: Any = None,
):
    """Decorator that ensures that only one of the arguments is passed.

    Parameters
    ----------
    exclusive_arg_names : Iterifiable[str], optional
        list of argument names that are exclusive, by default [], must conform to argument names of the decorated function
    may_be_empty : bool, optional
        whether the list of exclusive arguments passed to function can be empty, by default False
    empty_value : Any, optional
        value that is considered empty, by default None

    Returns
    -------
    Callable
        decorated function

    Raises
    ------
    ValueError
        if more than one of the exclusive arguments is passed
    ValueError
        if none of the exclusive arguments is passed and may_be_empty is False
    """

    exclusive_arg_names: list[str] = listify(exclusive_arg_names)

    def decorator(func):
        parameters = inspect.signature(func).parameters
        may_be_positional_arg_names = [
            name
            for name, param in parameters.items()
            if param.kind == param.POSITIONAL_OR_KEYWORD
        ]
        positions_of_exclusive_arg_names = set(
            [
                ind
                for ind, name in enumerate(may_be_positional_arg_names)
                if name in exclusive_arg_names
            ]
        )

        def wrapper(*args, **kwargs):
            count = 0
            for ind, arg in enumerate(args):
                if ind in positions_of_exclusive_arg_names and arg != empty_value:
                    count += 1
            for arg_name, arg in kwargs.items():
                if arg_name in exclusive_arg_names and arg != empty_value:
                    count += 1
            if not may_be_empty and count == 0:
                raise ValueError(
                    "Exactly one of the arguments must be passed, but no exclusive argument provided from: "
                    + ", ".join(exclusive_arg_names)
                )
            if count > 1:
                raise ValueError(
                    "Only one of the arguments can be passed from: "
                    + ", ".join(exclusive_arg_names)
                )
            return func(*args, **kwargs)

        return wrapper

    return decorator

#### Exclusive arguments examples


In [ ]:
@exclusive_args(["exc_a", "exc_d"])
def foo(exc_a, b=None, c=5, exc_d=None):
    print(exc_a, b, c, exc_d)

Only `exc_d` is used, and `exc_a` explicitly set to `None`


In [ ]:
foo(None, b=2, exc_d=3)

None 2 5 3


`exc_a` is used, and `exc_d` implicitly set to `None`, by default


In [ ]:
foo(3, b=2)

3 2 5 None


Explicitly set `exc_a` to `None`, and by default `exc_d` is `None`. In decorator we used `may_be_empty`=`False`, therefore raising an error


In [ ]:
# will raise ValueError
test_fail(
    lambda: foo(None),
    contains="Exactly one of the arguments must be passed, but no exclusive argument provided from: exc_a, exc_d",
)

Other fail cases


In [ ]:
# will raise ValueError
test_fail(
    lambda: foo(3, b=2, exc_d=3),
    contains="Only one of the arguments can be passed from: exc_a, exc_d",
)

In [ ]:
# will raise ValueError
test_fail(
    lambda: foo(3, 2, 5, 7),
    contains="Only one of the arguments can be passed from: exc_a, exc_d",
)

We could use it to convert objects to iterables


In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()